# Import All Necessary Modules And Setup Project

If you get any errors when importing these, ensure you run the commands:
```bash
$ python -m pip install -r requirements.txt
```
to install all necessary modules for this project. This command must be run from inside of this project directory.

It is recommended to use virtual environments for this project to ensure there is no conflicting package versions on your system.

Activate the virtual environment (if needed), run the pip install command, and then launch Jupyter Lab inside this project to get this project running.

In [ ]:
# Uncomment the following line to execute the pip install
# %pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns

from measure_incremental_development.compute import calculate_mid, classify_snapshots


## Get DF Representing Single Student Submission And File

In [ ]:
from getSubmissionDataframes import *

`getSubmissionDataframes` contains the following functions:

*   `getFileInStudentSubmission`
*   `getStudentSubmission`
*   `filterDownToRunAndEdits`
*   `filterDownToRunAndEditsAndPastes`
*   `getStudentSubmissionRunsAndEdits`
*   `getFileInStudentSubmissionRunsAndEdits`

## Reconstruct Submissions

In [ ]:
from reconstructSubmissions import *

`reconstructSubmissions` has the functions:

*   `reconstructSingleFileDebugger`
*   `reconstructFinalFile`
*   `reconstructFileAtRunEvents`
*   `reconstructProjectAtRunEvents`

#### View Reconstructed Project

In [ ]:
from viewReconstructions import *

`viewReconstructions` has the following functions:

*   `viewFinalReconstructedProject`
*   `viewReconstructedProjectStates`

#### Get Student Project Info

In [ ]:
from getStudentProjectInfo import *

`getStudentProjectInfo` has the following function:

*   `getStudentProjectList`

#### Filter Out Bad Reconstructions

In [ ]:
from filterOutBadReconstructions import *

`filterOutBadReconstructions` has the following functions:

*   `getFileReconstructionDF`
    *   Get the raw file reconstructions dataframe
*   `getProjectReconstructionDF`
    *   Same as above, but if any file in a project fails at reconstructing, the whole project is marked as a failed reconstruction
*   `getOnlyBadFileReconstructionsDF`
    *   Get a DF like `getFileReconstructionDF` returns, containing *ONLY* the bad file reconstructions
*   `getOnlyBadProjectReconstructions`
    *   Get a DF like `getProjectReconstructionDF` returns, containing *ONLY* the bad project reconstructions
*   `mergeKeystrokesWithFileReconstructions`
    *   Merge a keystroke dataframe with the *file* reconstruction df on `SubjectID, AssignmentID, CodeStateSection`
*   `mergeKeystrokesWithProjectReconstructions`
    *   Merge a keystroke dataframe with the *project* reconstruction df on `SubjectID, AssignmentID`
*   `getKeystrokesDFWithoutBadFileReconstructions`
    *   Filter down the keystroke dataframe to remove information related to *files* that reconstruct incorrectly
*   `getKeystrokesDFWithoutBadProjectReconstructions`
    *   Filter down the keystroke dataframe to remove information related to *projects* that reconstruct incorrectly

Unless granularity of the keystroke data is desired, the `getKeystrokesDFWithoutBadProjectReconstructions` will probably be the only needed function.

NOTE: The reconstruction data used for these functions was generated by the `checkSubmissions.sh` script and `determineReconstructionFailures.ipynb` notebook.

## Load Datasets

In [ ]:
keystroke_df_unedited = pd.read_csv("data/keystrokes.csv")
student_df_unedited = pd.read_csv("data/students.csv")

#### Copy Datasets For Modification

This preserves the initial datasets, in case we ever need to bring an unedited column/row back into anything

In [ ]:
keystroke_df = keystroke_df_unedited.copy()
student_df = student_df_unedited.copy()

#### Filter Keystroke Data To Only Projects That Have Reconstructed Correctly

In [ ]:
keystroke_df = getKeystrokesDFWithoutBadProjectReconstructions(keystroke_df)

#### Get Information And Keystroke Dataframes For Each `Student,Project` pair

**NOTE:** This may take a few minutes to compute. 

In [ ]:
projects_df, run_events_df, final_data = getStudentProjectList(student_df, keystroke_df)

print(len(projects_df), len(run_events_df))

In [ ]:
# List all students with a submission for the assignment
for student, assign, df in final_data:
    if len(df) > 0:
        print(student, assign)

In [ ]:
for student, assign, df in final_data:
    if len(df) > 0:
        # print(len(df))
        print(50*'=')
        print(student, assign)
        viewFinalReconstructedProject(df)
        print(50*'=')


## Add MID Library

- 0-2 Likely Incremental
- 2-2.5 Somewhat Incremental
- 2.5-3 Somewhat Non-Incremental
- 3+ Likely Non-Incremental

#### Calculate MID statistc for student and assignmemt

In [ ]:
from projectConstants import *
from midScoreFunctions import *

In [ ]:
print(get_scores('Student10', 'Assign10', student_df))

In [ ]:
mid_df = get_mid_score_all(final_data, student_df)

In [ ]:
mid_df.to_csv('./data/mid_scores.csv', index=False)

## Code to get the time between runs

In [ ]:
DATE_TIME_KEY = 'DateTime'
NEXT_DATE_TIME_KEY = 'NextDateTime'
DIFF_KEY = 'Diff'
DAYS_DIFF_KEY = 'DaysDiff'
HOURS_DIFF_KEY = 'HoursDiff'
MINUTES_DIFF_KEY = 'MinutesDiff'
SECONDS_DIFF_KEY = 'SecondsDiff'
SESSION_ID_KEY = 'SessionID'
SESSION_TIME_KEY = 'SessionTimeMs'
TOTAL_ASSIGNMENT_TIME_KEY = 'TotalAssignmentTimeMs'

In [ ]:
def getTimestampRow(row):
    '''Get a dictionary containing information for date time'''
    return {
        SUBJECT_ID_KEY: row[SUBJECT_ID_KEY],
        ASSIGNMENT_ID_KEY: row[ASSIGNMENT_ID_KEY],
        CLIENT_TIMESTAMP_KEY: row[CLIENT_TIMESTAMP_KEY],
        DATE_TIME_KEY: pd.to_datetime(row.ClientTimestamp, unit='ms')       
    }

In [ ]:
def getFilteredRunEvents(df):
    '''
    Get run events with execution action and start events only
    '''
    runDf = pd.DataFrame()
    
    for _, row in df.iterrows():
        if row.EventType == 'Run.Program':
            if row['X-Metadata'] != 'Start':
                continue
            timeRow = pd.DataFrame(getTimestampRow(row), index=[0])
            runDf = pd.concat([runDf, timeRow], ignore_index=True)
    return runDf

In [ ]:
runEvents = getFilteredRunEvents(keystroke_df)

In [ ]:
def getTimeBetweenRuns(df, student, assignment):
    '''Get time between runs for a student and assignment'''
    studentRunsDf = df[(df.SubjectID == student)&(df.AssignmentID == assignment)]
    studentRunsDf.sort_values(by=DATE_TIME_KEY, inplace=True)
    studentRunsDf[NEXT_DATE_TIME_KEY] = studentRunsDf[DATE_TIME_KEY].shift(-1)
    studentRunsDf[DIFF_KEY] = studentRunsDf[NEXT_DATE_TIME_KEY] - studentRunsDf[DATE_TIME_KEY]
    studentRunsDf[DAYS_DIFF_KEY] = round((studentRunsDf[NEXT_DATE_TIME_KEY] - studentRunsDf[DATE_TIME_KEY]).dt.days)
    studentRunsDf[HOURS_DIFF_KEY] = round((studentRunsDf[NEXT_DATE_TIME_KEY] - studentRunsDf[DATE_TIME_KEY]).dt.seconds / 3600.0, 2)
    studentRunsDf[MINUTES_DIFF_KEY] = round((studentRunsDf[NEXT_DATE_TIME_KEY] - studentRunsDf[DATE_TIME_KEY]).dt.seconds / 60.0, 2)
    studentRunsDf[SECONDS_DIFF_KEY] = round((studentRunsDf[NEXT_DATE_TIME_KEY] - studentRunsDf[DATE_TIME_KEY]).dt.seconds, 2)
    return studentRunsDf

In [ ]:
def getTimeBetweenRunsDf(keystroke_df, final_data):
    '''Get time between runs for each student'''
    runEvents = getFilteredRunEvents(keystroke_df)
    timeDifferenceDf = pd.DataFrame()
    for student, assignment, _ in final_data:
        studentDf = getTimeBetweenRuns(runEvents, student, assignment)
        timeDifferenceDf = pd.concat([timeDifferenceDf, studentDf], ignore_index=True)
    return timeDifferenceDf

In [ ]:
timeBetweenRunsDf = getTimeBetweenRunsDf(keystroke_df, final_data)

In [ ]:
display(timeBetweenRunsDf)
timeBetweenRunsDf.to_csv('./data/timeBetweenRuns.csv')

## Get Coding Sessions
#### Defined as keypresses within 5 minutes of eachother

In [ ]:
# use complete keystroke data
# student, assignment, number of coding sessions

def getCodingSessionsForAssignment(df, student, assignment):
    '''coding sessions end after more than 5 minutes from last event'''
    studentAssignmentDf = df[(df.SubjectID == student)&(df.AssignmentID == assignment)]
    # studentAssignmentDf = studentAssignmentDf[SUBJECT_ID_KEY, ASSIGNMENT_ID_KEY, CLIENT_TIMESTAMP_KEY]
    # convert client timestamp to date time
    studentAssignmentDf[DATE_TIME_KEY] = pd.to_datetime(studentAssignmentDf.ClientTimestamp, unit='ms') 
    studentAssignmentDf[SESSION_ID_KEY] = -1
    # sort by timestamp
    studentAssignmentDf.sort_values(by=DATE_TIME_KEY, inplace=True)
    # give session number , if within 5 minutes, keep that session number numbers
    if studentAssignmentDf.size > 0: 
        lastEventTime = studentAssignmentDf.head(1)[DATE_TIME_KEY].values[0]
    # 5 minutes from last event, not start of the session
    sessionId = 0
    for i, row in studentAssignmentDf.iterrows():
        diff = row[DATE_TIME_KEY] - lastEventTime
        minutes_diff = diff.total_seconds() / 60
        if minutes_diff > 5:
            sessionId += 1
        studentAssignmentDf.at[i,SESSION_ID_KEY] = sessionId
        lastEventTime = row[DATE_TIME_KEY]
    return studentAssignmentDf 

In [ ]:
def getTimeSpentPerSession(df):
    '''Get total time spent per session'''
    codingSessionDf = pd.DataFrame()
    sessionIds = df[SESSION_ID_KEY].unique()
    for sessionId in sessionIds:
        sessionDf = df[(df[SESSION_ID_KEY] == sessionId)]
        # sort by dates just in case
        sessionDf.sort_values(by=DATE_TIME_KEY, inplace=True)
        # subtract start session time and end session time to get total session time
        startTime = sessionDf.head(1)[CLIENT_TIMESTAMP_KEY].values[0]
        endTime = sessionDf.tail(1)[CLIENT_TIMESTAMP_KEY].values[0]
        sessionTime = endTime - startTime
        sessionDf[SESSION_TIME_KEY] = sessionTime
        codingSessionDf = pd.concat([codingSessionDf, sessionDf], ignore_index=True)
    return codingSessionDf

In [ ]:
def getTimeSpentPerAssignment(df):
    '''get total time spent per assignment using coding session time'''
    assignmentTimeDf = pd.DataFrame()
    assignments = df[ASSIGNMENT_ID_KEY].unique()
    for assignment in assignments:
        assignmentDf = df[(df[ASSIGNMENT_ID_KEY] == assignment)]
        # get unique session times
        sessionTimes = assignmentDf[SESSION_TIME_KEY].unique()
        # add all times in list
        totalAssignmentTime = sum(sessionTimes)
        assignmentDf[TOTAL_ASSIGNMENT_TIME_KEY] = totalAssignmentTime
        assignmentTimeDf = pd.concat([assignmentTimeDf, assignmentDf], ignore_index=True)
    return assignmentTimeDf

In [ ]:
def getCodingSessionsDf(keystroke_df, final_data):
    '''Get coding session for each student and assignment, where coding session is within 5 minutes'''
    runEvents = getFilteredRunEvents(keystroke_df)
    codingSessionDf = pd.DataFrame()
    for student, assignment, _ in final_data:
        codingSessionAssignmentDf = getCodingSessionsForAssignment(runEvents, student, assignment)
        # get time on assignment
        # time spent per session
        codingSessionTimeDf = getTimeSpentPerSession(codingSessionAssignmentDf)
        # total time spent per assignment
        if codingSessionTimeDf.size > 0:
            codingSessionTimeTotalDf = getTimeSpentPerAssignment(codingSessionTimeDf)
            codingSessionDf = pd.concat([codingSessionDf, codingSessionTimeTotalDf], ignore_index=True)
    return codingSessionDf

In [ ]:
codingSessions = getCodingSessionsDf(keystroke_df, final_data)

In [ ]:
codingSessions.to_csv('./data/codingSessions.csv')